In [1]:
import os
import glob
import random
import json
from PIL import Image
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision.transforms as T
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

# -----------------------------
# Set seed for reproducibility
# -----------------------------
random.seed(42)
torch.manual_seed(42)

# -----------------------------
# Path and File Collection
# -----------------------------
image_dir = "/kaggle/input/treesattfg/"
external_test_file = "/kaggle/input/treetecmodels/test_files.txt"

# Cargar todos los archivos .tif
all_files = glob.glob(f"{image_dir}/**/*.tif", recursive=True)
print("Number of images found:", len(all_files))
if len(all_files) == 0:
    raise ValueError("No .tif files found. Check the folder structure!")

# Cargar test_files.txt
with open(external_test_file, 'r') as f:
    test_filenames = [line.strip() for line in f.readlines()]

# Crear diccionario para buscar rutas por nombre de archivo
basename_to_path = {os.path.basename(p): p for p in all_files}

# Rutas completas para el test set
test_files = [basename_to_path[name] for name in test_filenames if name in basename_to_path]

# Filtrar archivos que no están en el test set
trainval_files = [f for f in all_files if os.path.basename(f) not in test_filenames]

# Dividir en entrenamiento y validación
train_files, val_files = train_test_split(trainval_files, test_size=0.2, random_state=42)

print(f"Train: {len(train_files)}, Val: {len(val_files)}, Test: {len(test_files)}")

# -----------------------------
# Load class list from JSON
# -----------------------------
def load_species_list(json_path):
    with open(json_path, 'r') as f:
        return json.load(f)

species_list = load_species_list("/kaggle/input/treesattfg/species_list.json")
species_list = sorted(species_list)

# -----------------------------
# Extract species from filename
# -----------------------------
def extract_species(filename):
    basename = os.path.basename(filename)
    parts = basename.split('_')
    species_parts = []
    for part in parts:
        if part.isdigit():
            break
        species_parts.append(part)
    species_name = "_".join(species_parts)
    if species_name.endswith("_spec."):
        species_name = species_name.split("_")[0]
    return species_name

# -----------------------------
# Dataset Definition
# -----------------------------
class TreeSpeciesDataset(Dataset):
    def __init__(self, files, transform=None):
        self.image_files = files
        self.transform = transform
        self.species_to_idx = {s: i for i, s in enumerate(species_list)}
        self.idx_to_species = {v: k for k, v in self.species_to_idx.items()}
        self.labels = [extract_species(f) for f in self.image_files]
        self.targets = [self.species_to_idx[label] for label in self.labels]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image = Image.open(self.image_files[idx]).convert("RGB")
        label = self.targets[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# -----------------------------
# Transform using EfficientNet weights
# -----------------------------
weights = EfficientNet_B0_Weights.DEFAULT
transform = weights.transforms()

# -----------------------------
# Create Datasets
# -----------------------------
train_dataset = TreeSpeciesDataset(train_files, transform=transform)
val_dataset = TreeSpeciesDataset(val_files, transform=transform)
test_dataset = TreeSpeciesDataset(test_files, transform=transform)

# -----------------------------
# Create DataLoaders
# -----------------------------
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# -----------------------------
# Save sample label mappings
# -----------------------------
output_path = "/kaggle/working/species_sample.txt"
with open(output_path, "w") as f:
    for i in range(0, len(train_dataset)):
        file = train_dataset.image_files[i]
        label_idx = train_dataset.targets[i]
        label_name = train_dataset.labels[i]
        line = f"{os.path.basename(file)} --> {label_name} (Index: {label_idx})\n"
        f.write(line)

Number of images found: 50381
Train: 34258, Val: 8565, Test: 7558


In [2]:
# -----------------------------
# Model Setup
# -----------------------------
print("🔧 Setting up model...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

num_classes = len(train_dataset.species_to_idx)
print(f"🔢 Number of classes: {num_classes}")

model = efficientnet_b0(weights=weights)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)
print("✅ EfficientNet-B0 model created and moved to device.")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
print("✅ Loss function, optimizer, and scheduler initialized.")

# -----------------------------
# Train and Evaluation Functions
# -----------------------------
def train(model, loader):
    print("🚂 Training...")
    model.train()
    total_loss = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print("✅ Training step completed.")
    return total_loss / len(loader)

def evaluate(model, loader):
    print("🔍 Evaluating...")
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    print("✅ Evaluation step completed.")
    return correct / total

# -----------------------------
# Training Loop
# -----------------------------
print("🏁 Starting training loop...")
EPOCHS = 20
for epoch in range(EPOCHS):
    print(f"\n📅 Epoch {epoch + 1}/{EPOCHS}")
    train_loss = train(model, train_loader)
    val_acc = evaluate(model, val_loader)
    scheduler.step()
    print(f"📊 Epoch {epoch+1} Results -> Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.4f}")

# -----------------------------
# Save Trained Model
# -----------------------------
print("\n💾 Saving model...")
torch.save(model.state_dict(), "efficientnetb0_tree_species.pth")
print("✅ Model saved as 'efficientnetb0_tree_species.pth'")

# -----------------------------
# Final Test Accuracy
# -----------------------------
print("\n🧪 Evaluating final test accuracy...")
test_acc = evaluate(model, test_loader)
print(f"🎯 Final Test Accuracy: {test_acc:.4f}")

🔧 Setting up model...
✅ Using device: cuda
🔢 Number of classes: 20


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 149MB/s]


✅ EfficientNet-B0 model created and moved to device.
✅ Loss function, optimizer, and scheduler initialized.
🏁 Starting training loop...

📅 Epoch 1/20
🚂 Training...
✅ Training step completed.
🔍 Evaluating...
✅ Evaluation step completed.
📊 Epoch 1 Results -> Train Loss: 1.5307 | Val Acc: 0.6718

📅 Epoch 2/20
🚂 Training...
✅ Training step completed.
🔍 Evaluating...
✅ Evaluation step completed.
📊 Epoch 2 Results -> Train Loss: 1.0454 | Val Acc: 0.6994

📅 Epoch 3/20
🚂 Training...
✅ Training step completed.
🔍 Evaluating...
✅ Evaluation step completed.
📊 Epoch 3 Results -> Train Loss: 0.8513 | Val Acc: 0.7122

📅 Epoch 4/20
🚂 Training...
✅ Training step completed.
🔍 Evaluating...
✅ Evaluation step completed.
📊 Epoch 4 Results -> Train Loss: 0.7057 | Val Acc: 0.7276

📅 Epoch 5/20
🚂 Training...
✅ Training step completed.
🔍 Evaluating...
✅ Evaluation step completed.
📊 Epoch 5 Results -> Train Loss: 0.5670 | Val Acc: 0.7302

📅 Epoch 6/20
🚂 Training...
✅ Training step completed.
🔍 Evaluating...
✅ E